# `attrs` by Example
---

## Basics

The simplest possible usage is:

In [4]:
import attr

In [5]:
@attr.s
class Empty(object):
    pass

In [6]:
Empty()

Empty()

In [7]:
Empty() == Empty()

True

In [8]:
Empty() is Empty()

False

So in other words: attrs is useful even without actual attributes!

But you'll usually want some data on your classes, so let's add some:

In [9]:
@attr.s
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()

By default, all features are added, so you immediately have a fully functional data class with a nice repr string and comparison methods.

In [15]:
c1 = Coordinates(1, 2)
c1

Coordinates(x=1, y=2)

In [16]:
c2 = Coordinates(x=2, y=1)
c2

Coordinates(x=2, y=1)

In [17]:
c1 == c2

False

As shown, the generated __init__ method allows for both positional and keyword arguments.

If playful naming turns you off, attrs comes with serious-business aliases:

In [18]:
from attr import attrs, attrib

@attrs
class SeriousCoordinates(object):
    x = attrib()
    y = attrib()

In [19]:
SeriousCoordinates(1, 2)

SeriousCoordinates(x=1, y=2)

In [20]:
attr.fields(Coordinates) == attr.fields(SeriousCoordinates)

True

For private attributes, attrs will strip the leading underscores for keyword arguments:

In [21]:
@attr.s
class C(object):
    _x = attr.ib()

In [22]:
C(x=1)

C(_x=1)

If you want to initialize your private attributes yourself, you can do that too:

In [23]:
@attr.s
class C(object):
    _x = attr.ib(init=False, default=42)

In [24]:
C()

C(_x=42)

In [26]:
C(23)

TypeError: __init__() takes 1 positional argument but 2 were given

In [27]:
class SomethingFromSomeoneElse(object):
    def __init__(self, x):
        self.x = x

In [28]:
SomethingFromSomeoneElse = attr.s(
    these={
        "x": attr.ib()
    }, init=False)(SomethingFromSomeoneElse)

In [29]:
SomethingFromSomeoneElse(1)

SomethingFromSomeoneElse(x=1)

Subclassing is bad for you, but attrs will still do what you'd hope for:

In [30]:
@attr.s
class A(object):
    a = attr.ib()
    def get_a(self):
        return self.a

In [31]:
@attr.s
class B(object):
    b = attr.ib()

In [32]:
@attr.s
class C(A, B):
    c = attr.ib()

In [33]:
i = C(1, 2, 3)
i

C(a=1, b=2, c=3)

In [34]:
i == C(1, 2, 3)

True

In [35]:
i.get_a()

1

The order of the attributes is defined by the MRO.

In Python 3, classes defined within other classes are detected and reflected in the __repr__. In Python 2 though, it's impossible. Therefore @attr.s comes with the repr_ns option to set it manually:

In [36]:
@attr.s
class C(object):
    @attr.s(repr_ns="C")
    class D(object):
        pass

In [37]:
C.D()

C.D()

repr_ns works on both Python 2 and 3. On Python 3 it overrides the implicit detection.

Keyword-only Attributes
When using attrs on Python 3, you can also add keyword-only attributes:

In [ ]:
@attr.s
class A:
    a = attr.ib(kw_only=True)

In [ ]:
A()

In [48]:
A(a=1)

A(a=1)

kw_only may also be specified at via attr.s, and will apply to all attributes:

In [ ]:
@attr.s(kw_only=True)
class A:
    a = attr.ib()
    b = attr.ib()

In [50]:
A(1, 2)

TypeError: __init__() takes 2 positional arguments but 3 were given

In [52]:
A(a=1, b=2)

TypeError: __init__() got an unexpected keyword argument 'b'

If you create an attribute with init=False, the kw_only argument is ignored.

Keyword-only attributes allow subclasses to add attributes without default values, even if the base class defines attributes with default values:

In [53]:
@attr.s
class A:
    a = attr.ib(default=0)

In [54]:
@attr.s
class B(A):
    b = attr.ib(kw_only=True)

TypeError: attrib() got an unexpected keyword argument 'kw_only'

In [55]:
B(b=1)

B(b=1)

In [56]:
B()

TypeError: __init__() missing 1 required positional argument: 'b'

If you don't set kw_only=True, then there's is no valid attribute ordering and you'll get an error:

In [57]:
@attr.s
class A:
    a = attr.ib(default=0)

In [58]:
@attr.s
class B(A):
    b = attr.ib()

ValueError: No mandatory attributes allowed after an attribute with a default value or factory.  Attribute in question: Attribute(name='b', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None)

## Converting to Collections Types

When you have a class with data, it often is very convenient to transform that class into a :class:`dict` (for example if you want to serialize it to JSON):

In [59]:
attr.asdict(Coordinates(x=1, y=2))

{'x': 1, 'y': 2}

Some fields cannot or should not be transformed. For that, :func:`attr.asdict` offers a callback that decides whether an attribute should be included:

In [60]:
@attr.s
class UserList(object):
    users = attr.ib()

In [61]:
@attr.s
class User(object):
    email = attr.ib()
    password = attr.ib()

In [63]:
attr.asdict(
    UserList([
        User("jane@doe.invalid", "s33kred"),
        User("joe@doe.invalid", "p4ssw0rd")
    ]),
    filter=lambda attr, value: attr.name != "password"
)

{'users': [{'email': 'jane@doe.invalid'}, {'email': 'joe@doe.invalid'}]}

For the common case where you want to :func:`include <attr.filters.include>` or :func:`exclude <attr.filters.exclude>` certain types or attributes, attrs ships with a few helpers:

In [64]:
@attr.s
class User(object):
    login = attr.ib()
    password = attr.ib()
    id = attr.ib()

In [65]:
attr.asdict(
    User("jane", "s33kred", 42),
    filter=attr.filters.exclude(attr.fields(User).password, int)
)

{'login': 'jane'}

In [66]:
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()
    z = attr.ib()

In [67]:
attr.asdict(
    C("foo", "2", 3),
    filter=attr.filters.include(int, attr.fields(C).x)
)

{'x': 'foo', 'z': 3}

Other times, all you want is a tuple and attrs won't let you down:

In [68]:
import sqlite3
import attr

@attr.s
class Foo:
    a = attr.ib()
    b = attr.ib()

In [69]:
foo = Foo(2, 3)
with sqlite3.connect(":memory:") as conn:
    c = conn.cursor()
    c.execute("CREATE TABLE foo (x INTEGER PRIMARY KEY ASC, y)") #doctest: +ELLIPSIS
    c.execute("INSERT INTO foo VALUES (?, ?)", attr.astuple(foo)) #doctest: +ELLIPSIS
    foo2 = Foo(*c.execute("SELECT x, y FROM foo").fetchone())

In [70]:
foo == foo2

True

## Defaults

Sometimes you want to have default values for your initializer. And sometimes you even want mutable objects as default values (ever used accidentally def f(arg=[])?). attrs has you covered in both cases:

In [72]:
import collections

@attr.s
class Connection(object):
    socket = attr.ib()
    @classmethod
    def connect(cls, db_string):
        # ... connect somehow to db_string ...
        return cls(socket=42)

In [73]:
@attr.s
class ConnectionPool(object):
    db_string = attr.ib()
    pool = attr.ib(default=attr.Factory(collections.deque))
    debug = attr.ib(default=False)

    def get_connection(self):
        try:
            return self.pool.pop()
        except IndexError:
            if self.debug:
                print("New connection!")
            return Connection.connect(self.db_string)
    def free_connection(self, conn):
        if self.debug:
            print("Connection returned!")
        self.pool.appendleft(conn)

In [74]:
cp = ConnectionPool("postgres://localhost")
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([]), debug=False)

In [75]:
conn = cp.get_connection()
conn

Connection(socket=42)

In [76]:
cp.free_connection(conn)
cp

ConnectionPool(db_string='postgres://localhost', pool=deque([Connection(socket=42)]), debug=False)

More information on why class methods for constructing objects are awesome can be found in this insightful blog post.

Default factories can also be set using a decorator. The method receives the partially initialized instance which enables you to base a default value on other attributes:

In [77]:
@attr.s
class C(object):
    x = attr.ib(default=1)
    y = attr.ib()
    @y.default
    def name_does_not_matter(self):
        return self.x + 1

In [78]:
C()

C(x=1, y=2)

And since the case of attr.ib(default=attr.Factory(f)) is so common, attrs also comes with syntactic sugar for it:

In [79]:
@attr.s
class C(object):
    x = attr.ib(factory=list)

In [80]:
C()

C(x=[])

## Validators

Although your initializers should do as little as possible (ideally: just initialize your instance according to the arguments!), it can come in handy to do some kind of validation on the arguments.

attrs offers two ways to define validators for each attribute and it's up to you to choose which one suits your style and project better.

You can use a decorator:

In [81]:
@attr.s
class C(object):
    x = attr.ib()
    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("x must be smaller or equal to 42")

In [82]:
C(42)

C(x=42)

In [83]:
C(43)

ValueError: x must be smaller or equal to 42

In [84]:
def x_smaller_than_y(instance, attribute, value):
    if value >= instance.y:
        raise ValueError("'x' has to be smaller than 'y'!")

In [85]:
@attr.s
class C(object):
    x = attr.ib(validator=[
        attr.validators.instance_of(int), x_smaller_than_y])
    y = attr.ib()

In [86]:
C(x=3, y=4)

C(x=3, y=4)

In [87]:
C(x=4, y=3)

ValueError: 'x' has to be smaller than 'y'!

In [88]:
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.instance_of(int))
    @x.validator
    def fits_byte(self, attribute, value):
        if not 0 <= value < 256:
            raise ValueError("value out of bounds")

In [89]:
C(128)

C(x=128)

In [90]:
C("128")

TypeError: ("'x' must be <class 'int'> (got '128' that is a <class 'str'>).", Attribute(name='x', default=NOTHING, validator=_AndValidator(_validators=(<instance_of validator for type <class 'int'>>, <function C.fits_byte at 0x104e76158>)), repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None), <class 'int'>, '128')

In [91]:
C(256)

ValueError: value out of bounds

In [92]:
@attr.s
class C(object):
    x = attr.ib(validator=attr.validators.instance_of(int))

In [93]:
C(42)

C(x=42)

In [94]:
C("42")

TypeError: ("'x' must be <class 'int'> (got '42' that is a <class 'str'>).", Attribute(name='x', default=NOTHING, validator=<instance_of validator for type <class 'int'>>, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None), <class 'int'>, '42')

## Conversion

Attributes can have a converter function specified, which will be called with the attribute's passed-in value to get a new value to use. This can be useful for doing type-conversions on values that you don't want to force your callers to do.

In [95]:
@attr.s
class C(object):
    x = attr.ib(converter=int)

In [96]:
o = C("1")
o.x

1

Check out `converters` for more details.

## Metadata

All attrs attributes may include arbitrary metadata in the form of a read-only dictionary.

In [97]:
@attr.s
class C(object):
    x = attr.ib(metadata={'my_metadata': 1})

In [98]:
attr.fields(C).x.metadata

mappingproxy({'my_metadata': 1})

In [99]:
attr.fields(C).x.metadata['my_metadata']

1

Metadata is not used by attrs, and is meant to enable rich functionality in third-party libraries. The metadata dictionary follows the normal dictionary rules: keys need to be hashable, and both keys and values are recommended to be immutable.

If you're the author of a third-party library with attrs integration, please see :ref:`Extending Metadata <extending_metadata>`.

## Types

attrs also allows you to associate a type with an attribute using either the type argument to :func:`attr.ib` or -- as of Python 3.6 -- using PEP 526-annotations:

In [100]:
@attr.s
class C:
    x = attr.ib(type=int)
    y: int = attr.ib()

In [101]:
attr.fields(C).x.type

int

In [102]:
attr.fields(C).y.type

int

If you don't mind annotating all attributes, you can even drop the :func:`attr.ib` and assign default values instead:

In [103]:
import typing

@attr.s(auto_attribs=True)
class AutoC:
    cls_var: typing.ClassVar[int] = 5  # this one is ignored
    l: typing.List[int] = attr.Factory(list)
    x: int = 1
    foo: str = attr.ib(
         default="every attrib needs a type if auto_attribs=True"
    )
    bar: typing.Any = None

In [104]:
attr.fields(AutoC).l.type

typing.List[int]

In [105]:
attr.fields(AutoC).x.type

int

In [106]:
attr.fields(AutoC).foo.type

str

In [107]:
attr.fields(AutoC).bar.type

typing.Any

In [108]:
AutoC()

AutoC(l=[], x=1, foo='every attrib needs a type if auto_attribs=True', bar=None)

In [109]:
AutoC.cls_var

5

The generated __init__ method will have an attribute called __annotations__ that contains this type information.

Warning

attrs itself doesn't have any features that work on top of type metadata yet. However it's useful for writing your own validators or serialization frameworks.

## Slots

`Slotted classes` have a bunch of advantages on CPython. Defining __slots__ by hand is tedious, in attrs it's just a matter of passing slots=True:

In [110]:
@attr.s(slots=True)
class Coordinates(object):
    x = attr.ib()
    y = attr.ib()

## Immutability

Sometimes you have instances that shouldn't be changed after instantiation. Immutability is especially popular in functional programming and is generally a very good thing. If you'd like to enforce it, attrs will try to help:

In [111]:
@attr.s(frozen=True)
class C(object):
    x = attr.ib()

In [112]:
i = C(1)
i.x = 2

FrozenInstanceError: 

In [114]:
i.x

1

Please note that true immutability is impossible in Python but it will :ref:`get <how-frozen>` you 99% there. By themselves, immutable classes are useful for long-lived objects that should never change; like configurations for example.

In order to use them in regular program flow, you'll need a way to easily create new instances with changed attributes. In Clojure that function is called assoc and attrs shamelessly imitates it: `attr.evolve`:

In [115]:
@attr.s(frozen=True)
class C(object):
    x = attr.ib()
    y = attr.ib()

In [116]:
i1 = C(1, 2)
i1

C(x=1, y=2)

In [118]:
i2 = attr.evolve(i1, y=3)
i2

C(x=1, y=3)

In [120]:
i1 == i2

False

## Other Goodies

Sometimes you may want to create a class programmatically. attrs won't let you down and gives you :func:`attr.make_class` :

In [121]:
@attr.s
class C1(object):
    x = attr.ib()
    y = attr.ib()

In [122]:
C2 = attr.make_class("C2", ["x", "y"])
attr.fields(C1) == attr.fields(C2)

True

You can still have power over the attributes if you pass a dictionary of name: attr.ib mappings and can pass arguments to @attr.s:

In [123]:
C = attr.make_class(
    "C", {"x": attr.ib(default=42), "y": attr.ib(default=attr.Factory(list))},
    repr=False)

In [124]:
i = C()
i  # no repr added!

In [125]:
i.x

42

In [126]:
i.y

[]

If you need to dynamically make a class with `attr.make_class` and it needs to be a subclass of something else than object, use the bases argument:

In [127]:
class D(object):
    def __eq__(self, other):
        return True  # arbitrary example

In [128]:
C = attr.make_class("C", {}, bases=(D,), cmp=False)
isinstance(C(), D)

True

Sometimes, you want to have your class's __init__ method do more than just the initialization, validation, etc. that gets done for you automatically when using @attr.s. To do this, just define a __attrs_post_init__ method in your class. It will get called at the end of the generated __init__ method.

In [129]:
@attr.s
class C(object):
    x = attr.ib()
    y = attr.ib()
    z = attr.ib(init=False)
    def __attrs_post_init__(self):
        self.z = self.x + self.y

In [130]:
obj = C(x=1, y=2)
obj

C(x=1, y=2, z=3)

Finally, you can exclude single attributes from certain methods:

In [131]:
@attr.s
class C(object):
    user = attr.ib()
    password = attr.ib(repr=False)

In [132]:
C("me", "s3kr3t")

C(user='me')

## Do your own experiments here...

Try `attrs` youself by adding your code below and running your own experiments 👇

In [ ]:
import attr

# your code here
attr.